In [25]:
import os
import json

def load_json_files(folder_path):
    data = []
    for folder_name in os.listdir(folder_path):
        folder_full_path = os.path.join(folder_path, folder_name)
        if os.path.isdir(folder_full_path):
            print(folder_full_path)
            files = os.listdir(folder_full_path)

            for file_name in files:
                if file_name.endswith('.json'):
                    file_path = os.path.join(folder_full_path, file_name)
                    dataset = {
                        'model' : folder_name.replace("_image" , ""),
                        'dataset' : file_name.replace("_results.json" , ""),
                        'results' : None
                    }

                    with open(file_path, 'r') as f:
                        dataset['results'] = json.load(f)

                    data.append(dataset)
    return data

# Load all JSON files in the 'res' folder
folder_path = './res/'
data = load_json_files(folder_path)

for d in data:
    print(d['model'], d['dataset'], len(d['results']))
    print(d['results'][0])


./res/Unisal_finetune_true_threshold_None
./res/TranSalNetDense_finetune_true_threshold_None
./res/salicon_test_Unisal_threshold_None
./res/TranSalNetDense_finetune_true_threshold_0.8
./res/Unisal_finetune_true_threshold_0.9
Unisal_finetune_true_threshold_None mit1003 1003
{'filename': 'i1074021825.jpeg', 'path': '/Users/coconut/Documents/Dataset/Saliency/MIT1003/images', 'metrics': {'saliency': {'NSS': 0.529699981212616, 'CC': 0.5871, 'KLD': 0.8551, 'SIM': 0.5437}, 'rarity': {'NSS': 0.35670000314712524, 'CC': 0.5841, 'KLD': 1.481, 'SIM': 0.7807}, 'saliency_Add': {'NSS': 0.5188999772071838, 'CC': 0.702, 'KLD': 1.1702, 'SIM': 0.777}, 'saliency_Sub': {'NSS': 0.12470000237226486, 'CC': 0.3351, 'KLD': 2.0382, 'SIM': 0.705}, 'saliency_Prod': {'NSS': 0.4277999997138977, 'CC': 0.5176, 'KLD': 1.0688, 'SIM': 0.4509}, 'saliency_Itti': {'NSS': 0.5343999862670898, 'CC': 0.7045, 'KLD': 1.1232, 'SIM': 0.7714}}}
TranSalNetDense_finetune_true_threshold_None mit1003 101
{'filename': 'i1074021825.jpeg',

In [26]:
import numpy as np

# def calculate_mean(data,key):
#     msrts, msrbs = [], []
#     for item in data:
#         msrts.append(item['metrics'][key]['msrt'])
#         msrbs.append(item['metrics'][key]['msrb'])  
#     msrts = np.asarray(msrts)
#     msrbs = np.asarray(msrbs)
#     mean_msrt = msrts.sum() / len(msrts)
#     mean_msrb = msrbs.sum() / len(msrbs)
#     return float(round(mean_msrt,2)), float(round(mean_msrb,2))


def calculate_mean(data,key,mkey):
    m= []
    for item in data:
        if item['metrics'][key][mkey] is None or np.isnan(item['metrics'][key][mkey]):
            continue
        m.append(item['metrics'][key][mkey])

    if len(m) == 0:
        return None
    
    m = np.asarray(m)
    m = m.sum() / len(m)
    return float(round(m,2))


results = []
for d in data:
    # get list of all keys resulst
    print(d)
    sal_keys = list(d['results'][0]['metrics'].keys())


    for key in sal_keys:

        result= {
            'model' : d['model'],
            'dataset' : d['dataset'],
            'key' : key,
            'metrics' :  {
            }
        }   

        metrics_keys = list(d['results'][0]['metrics'][key].keys())
        for mkey in metrics_keys:
            mean__ = calculate_mean(d['results'],key,mkey)
            if mean__ is None:
                continue
            result['metrics'][mkey] = mean__


        

        print(result)

        results.append(result)

    # break

{'model': 'Unisal_finetune_true_threshold_None', 'dataset': 'mit1003', 'results': [{'filename': 'i1074021825.jpeg', 'path': '/Users/coconut/Documents/Dataset/Saliency/MIT1003/images', 'metrics': {'saliency': {'NSS': 0.529699981212616, 'CC': 0.5871, 'KLD': 0.8551, 'SIM': 0.5437}, 'rarity': {'NSS': 0.35670000314712524, 'CC': 0.5841, 'KLD': 1.481, 'SIM': 0.7807}, 'saliency_Add': {'NSS': 0.5188999772071838, 'CC': 0.702, 'KLD': 1.1702, 'SIM': 0.777}, 'saliency_Sub': {'NSS': 0.12470000237226486, 'CC': 0.3351, 'KLD': 2.0382, 'SIM': 0.705}, 'saliency_Prod': {'NSS': 0.4277999997138977, 'CC': 0.5176, 'KLD': 1.0688, 'SIM': 0.4509}, 'saliency_Itti': {'NSS': 0.5343999862670898, 'CC': 0.7045, 'KLD': 1.1232, 'SIM': 0.7714}}}, {'filename': 'i1486171792.jpeg', 'path': '/Users/coconut/Documents/Dataset/Saliency/MIT1003/images', 'metrics': {'saliency': {'NSS': 0.4893999993801117, 'CC': 0.5315, 'KLD': 1.0379, 'SIM': 0.5447}, 'rarity': {'NSS': 0.4814999997615814, 'CC': 0.6009, 'KLD': 1.2581, 'SIM': 0.7739}

In [27]:
from IPython.display import display, HTML
# get list of datasets from results
datasets_name = list(set([result['dataset'] for result in results]))
datasets_results = {dataset : [] for dataset in datasets_name}

print(datasets_results)

for result in results:
    datasets_results[result['dataset']].append(result)

import pandas as pd
for dataset, data in datasets_results.items():
    print(dataset)

    table_data = {
        'Models': [],
        'Type': [],
        # 'MSRT': msrts,
        # 'MSRB': msrbs
    }

    metrics= []
    for res in data:
        name = res['model'] + " " + res['key']

        table_data['Models'].append(res['model'])
        table_data['Type'].append(res['key'])

        for mk, mv in res['metrics'].items():
            if mk not in table_data:
                metrics.append(mk)
                table_data[mk] = []
            table_data[mk].append(mv)
    # show tables results of names msrts and msrbs

    df = pd.DataFrame(table_data)
    styled_df = df.style.background_gradient(subset=metrics, cmap='coolwarm')
    display(styled_df)




{'mit1003': []}
mit1003


,Models,Type,NSS,CC,KLD,SIM
0,Unisal_finetune_true_threshold_None,saliency,0.760000,0.550000,1.040000,0.510000
1,Unisal_finetune_true_threshold_None,rarity,0.660000,0.590000,1.430000,0.760000
2,Unisal_finetune_true_threshold_None,saliency_Add,0.810000,0.650000,1.160000,0.730000
3,Unisal_finetune_true_threshold_None,saliency_Sub,0.390000,0.380000,1.870000,0.680000
4,Unisal_finetune_true_threshold_None,saliency_Prod,0.610000,0.450000,1.110000,0.420000
5,Unisal_finetune_true_threshold_None,saliency_Itti,0.820000,0.650000,1.140000,0.720000
6,TranSalNetDense_finetune_true_threshold_None,saliency,1.000000,0.650000,0.980000,0.550000
7,TranSalNetDense_finetune_true_threshold_None,rarity,0.550000,0.390000,1.510000,0.710000
8,TranSalNetDense_finetune_true_threshold_None,saliency_Add,0.920000,0.620000,1.070000,0.710000
9,TranSalNetDense_finetune_true_threshold_None,saliency_Sub,0.470000,0.310000,1.530000,0.670000
